# Imports

## Libraries

In [ ]:
import random
import os
import time

import matplotlib

from ipywidgets import *
from IPython.display import display

device = "cpu"

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Import Utility Functions 

Importing from [utils.py](files/utils.py).

In [ ]:
from utils import *

## Import Image Processing Functions

Importing from [image_processing.py](files/image_processing.py).

In [ ]:
from image_processing import *

# Load Frames

## Load all frames

In [ ]:
folders = ["all_frames"] # os.listdir(dataset)

triplets_per_folder = {}
MAX = 15

for folder in folders:
    triplets_per_folder[folder] = []
    files = sorted(os.listdir(folder), key=lambda file: int(file.split("_")[0]))
    rgb_im_files = list(filter(lambda file: "rgb.png" in file, files))
    depth_im_files = list(filter(lambda file: "depth.png" in file, files))
    depth_arr_files = list(filter(lambda file: "depth.npy" in file, files))
    num_files = len(rgb_im_files)
    
    step = num_files // MAX if MAX != np.inf else 1 # TODO: Remove later
    idxs = range(0, num_files, step)
    
    for i in idxs:
        rgb_im_path = os.path.join(folder, rgb_im_files[i])
        depth_im_path = os.path.join(folder, depth_im_files[i])
        depth_arr_path = os.path.join(folder, depth_arr_files[i])

        rgb_img = cv2.imread(rgb_im_path, cv2.IMREAD_COLOR)     # BGR image
        depth_img = cv2.imread(depth_im_path, cv2.IMREAD_COLOR) # BGR image
        depth_arr = np.load(depth_arr_path)
        
        triplets_per_folder[folder].append((rgb_img, depth_img, depth_arr))
    
print("Number of folders:", len(triplets_per_folder))
for i, k in enumerate(folders):
    print("{}) Loaded {} frames from {}".format(i, len(triplets_per_folder[k]), k))

## Pick one frame

In [ ]:
# folder_indices = list(range(8))
# folder_idx = random.choice(folder_indices)
# print("Folder choice:", folder_idx)
folder_idx = 0
folder = folders[folder_idx]

# file_indices = list(range(0, 120))
# frame_idx = random.choice(file_indices)
# print("Frame choice:", frame_idx)
frame_idx = 15

print("Fetching frame: {} from folder: {}".format(frame_idx+1, folder))
rgb_img, depth_img, depth_arr = triplets_per_folder[folder][frame_idx]

show_image_pairs(rgb_img, depth_img)

# for i in range(len(triplets_per_folder["all_frames"])):
#     rgb_img, depth_img, depth_arr = triplets_per_folder[folder][i]

#     show_image_pairs(rgb_img, depth_img)

# Detect Corners & Edge

In [ ]:
template = create_template(size=30, position=1)

In [ ]:
# Fit a curve (2nd degree polynomial) to inlier detections
edge_pixels, inliers, outliers = apply_template_matching(depth_arr,
                                                         template,
                                                         start_depth=1.0,    # Given in depth-scale
                                                         contour_width=25,   # Given in y-scale
                                                         y_step=5,           # Given in y-scale
                                                         n_contours=1000,
                                                         dynamic_width=True,
                                                         ransac_thresh=15,
                                                         score_thresh=None,
                                                         roi=[None,None,250,None],
                                                         fit_type="curve",
                                                         verbose=3)

# Visualize inlier and outlier corners
plt.figure(figsize=(10,10))
plt.imshow(depth_img)
inlier_pts = plt.scatter(inliers[:,1], inliers[:,0], color="black", marker="o")
outlier_pts = plt.scatter(outliers[:,1], outliers[:,0], color="purple", marker="^")
plt.legend((inlier_pts, outlier_pts), ("inliers", "outliers"), loc=1)
plt.show()
# plt.savefig(f"out/pts_{frame_idx}.png")

# Visualize inlier and outlier corners and fitted curve
plt.figure(figsize=(10,10))
plt.imshow(depth_img)
plt.plot(edge_pixels[:,1], edge_pixels[:,0], color="cyan")
inlier_pts = plt.scatter(inliers[:,1], inliers[:,0], color="black", marker="o")
outlier_pts = plt.scatter(outliers[:,1], outliers[:,0], color="purple", marker="^")
plt.legend((inlier_pts, outlier_pts), ("inliers", "outliers"), loc=1)
plt.show()
# plt.savefig(f"out/pts_{frame_idx}.png")

# Create binary mask for the curve fitted
mask = np.zeros(depth_arr.shape, dtype=np.bool)
tmp = convert_coord2mask(depth_arr.shape, edge_pixels)
mask = mask | tmp
    
# Overlay the mask on original RGB
overlaid = rgb_img.copy()
color = (0, 0, 0) if len(overlaid.shape) == 3 else 255
overlaid[mask == 255] = color
plt.figure(figsize=(10,10))
plt.imshow(overlaid)
plt.show()
# plt.savefig(f"out/rgb_overlay_{frame_idx}.png")

# Overlay the mask on original depth
overlaid = depth_img.copy()
color = (0, 0, 0) if len(overlaid.shape) == 3 else 255
overlaid[mask == 255] = color
plt.figure(figsize=(10,10))
plt.imshow(overlaid)
plt.show()
# plt.savefig(f"out/depth_overlay_{frame_idx}.png")